<a href="https://colab.research.google.com/github/Foutse/My_NLP_Projects/blob/master/Article_summary_with_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Get summary and sentiment of a web article**
In an era where information overload is a common challenge, the ability to quickly understand and gauge the emotional tone of an article becomes invaluable. This notebook aims to address this need through a user-friendly widget. We guide you into the intriguing world of NLP by offering a comprehensive tool for summarizing and analyzing the sentiment of web articles.

## **What to Expect**

In this notebook, you will encounter an innovative widget designed to simplify the process of extracting meaningful insights from web-based articles.
Here’s a brief overview of what this notebook offers:
* **Web Article Summarization:** With just a few clicks, you can obtain concise summaries of lengthy web articles. This feature is particularly useful for those who need to quickly grasp the essence of an article without spending time reading the entire content.
* **Sentiment Analysis:** The notebook also provides sentiment analysis of the selected articles. This allows you to understand the underlying emotional tone – whether it's positive, negative, or neutral. This feature is invaluable for researchers, marketers, and anyone interested in public opinion and media analysis.
* **Interactive Widget:** At the heart of this notebook is an interactive widget that makes the entire process seamless and user-friendly. You simply input the URL of the web article, and the widget takes care of the rest, presenting you with both the summary and the sentiment analysis, with some few details about the article like its title, author, and publication date.

By the end of this notebook, you will have hands-on experience with a powerful NLP application that showcases the practical benefits of summarization and sentiment analysis in real-world scenarios. Enjoy!


# Some helpful links:
* https://python.langchain.com/docs/integrations/document_loaders/web_base
* https://python.langchain.com/docs/use_cases/summarization
* https://python.langchain.com/docs/modules/chains/document/stuff

# Installations of the neccessary libraries

In [ ]:
!pip install newspaper3k #https://newspaper.readthedocs.io/en/latest/
!pip install textblob==0.9.0 #https://textblob.readthedocs.io/en/dev/
!pip install nltk #https://www.nltk.org/

In [ ]:
!pip install langchain #https://www.langchain.com/
!pip install openai #https://openai.com/

In [ ]:
!pip install --upgrade pydantic #https://docs.pydantic.dev/latest/
!pip install --force-reinstall langchain

In [41]:
#@title Get neccesary libraries
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.summarize import load_summarize_chain
from langchain_community.chat_models import ChatOpenAI
from newspaper import Article,Config
from newspaper import ArticleException
from textblob import TextBlob
import ipywidgets as widgets
import nltk
import os
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [42]:
os.environ["OPENAI_API_KEY"] = "Your_OPENAI_API_KEY_goes_here"

In [43]:
#@title set up a custom configuration for the newspaper library with a modified user-agent:
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
config = Config()
config.browser_user_agent = user_agent


In [44]:
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo-1106")
chain = load_summarize_chain(llm, chain_type="stuff")

In [74]:
#@title Create the input field, output areas, and button:
layout_large = widgets.Layout(width='800px', height='200px')  # Large layout for text areas
layout_small = widgets.Layout(width='800px', height='50px')   # Smaller layout for single-line inputs
style = {'description_width': 'initial'}

utext = widgets.Textarea(description='URL:', layout=widgets.Layout(width='800px', height='20px'))
title = widgets.Textarea(description='Title:', disabled=True, layout=layout_small)
author = widgets.Textarea(description='Author:', disabled=True, layout=layout_small)
publication = widgets.Textarea(description='Pub. Date:', disabled=True, layout=layout_small)
summary = widgets.Textarea(description='Summary:', disabled=True, layout=layout_large)
sentiment = widgets.Textarea(description='Sentiment:', disabled=True, layout=layout_small)
button = widgets.Button(description='Search', layout=widgets.Layout(width='100px'))
button.style.button_color = 'black'


In [75]:
#@title Define the summarization function:
def summarise(b):
  button.description = 'Searching...'
  button.disabled = True
  try:
    url = utext.value.strip()
    loader = WebBaseLoader(url)
    docs = loader.load()
    article = Article(url,config=config)
    article.download()
    article.parse()
    article.nlp()

    title.value = article.title if article.title else '-Not Found-'
    author.value = ', '.join(article.authors) if article.authors else '-Not Found-'
    publication.value = str(article.publish_date) if article.publish_date else '-Not Found-'
    # Use Langchain to summarize
    summary_text = chain.run(docs)
    summary.value = summary_text if summary_text else article.text

    analysis = TextBlob(article.text) if article.text else TextBlob(str(docs))
    sentiment_val = "Positive" if analysis.polarity > 0 else "Negative" if analysis.polarity < 0 else "Neutral"
    sentiment.value = sentiment_val
  except ArticleException as e:
      summary.value = "Failed to download the article: " + str(e)
  finally:
    button.description = 'Summarized'
    button.disabled = False

button.on_click(summarise)


In [77]:
#@title Display widgets
utext.value = ''
title.value = ''
author.value = ''
publication.value = ''
summary.value = ''
sentiment.value = ''

# Then display the widgets
display(utext, button, title, author, publication, summary, sentiment)

Textarea(value='', description='URL:', layout=Layout(height='20px', width='800px'))

Button(description='Summarized', layout=Layout(width='100px'), style=ButtonStyle(button_color='black'))

Textarea(value='', description='Title:', disabled=True, layout=Layout(height='50px', width='800px'))

Textarea(value='', description='Author:', disabled=True, layout=Layout(height='50px', width='800px'))

Textarea(value='', description='Pub. Date:', disabled=True, layout=Layout(height='50px', width='800px'))

Textarea(value='', description='Summary:', disabled=True, layout=Layout(height='200px', width='800px'))

Textarea(value='', description='Sentiment:', disabled=True, layout=Layout(height='50px', width='800px'))

# Conclusion
The summarization and sentiment analysis features provide a dual advantage. Summarization condenses the content, making it easier to digest, while sentiment analysis adds a layer of understanding regarding the emotional tone of the text. This combination is powerful for a comprehensive understanding of web articles.

## **Importance of this Application**

The significance of this application lies in its ability to distill large amounts of text into digestible summaries and to provide a quick understanding of the article's emotional tone. In our fast-paced world, where time is of the essence, such a tool is incredibly beneficial for:
* **Students and Academics:** For those engaged in research, this tool can help in quickly navigating through a plethora of articles, focusing only on the relevant information.
* **Professionals:** For business professionals, especially those in communications, media, or public relations, understanding both the content and sentiment of various articles can be crucial for strategy development.
* **Casual Readers:** Even for the general public, this tool offers a quick way to stay informed without being overwhelmed by the volume of available information.

## **Implications and Future Directions**

* For Academics and Researchers: This tool can be a starting point for more advanced research in automated text summarization and sentiment analysis, potentially leading to more nuanced and accurate algorithms.

* In Professional Settings: Businesses and organizations can adapt this technology for media monitoring, brand sentiment analysis, and quick information retrieval, enhancing decision-making processes.

* Personal Use and Education: The tool can serve as an educational resource for those interested in NLP and its practical applications, fostering a deeper interest in AI and machine learning.

The field of NLP is continuously evolving, promising even more sophisticated tools and applications in the future. We hope that this notebook has not only provided you with a useful tool but also sparked your curiosity and interest in the potential of NLP and AI.
Thank you for joining me on this enlightening journey! :)

![Until Next Time!](https://media.tenor.com/4QWjoG02bPwAAAAC/until-next-time-next-time.gif)
